In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, ttk
import numpy as np
from transformers import MarianMTModel, MarianTokenizer

# Define available languages and their MarianMT model codes
LANGUAGE_CODES = {
    "English": "en",
    "Spanish": "es",
    "German": "de"
}

# MarianMT model mapping for translation pairs
TRANSLATION_MODELS = {
    ("en", "es"): "Helsinki-NLP/opus-mt-en-es",
    ("en", "de"): "Helsinki-NLP/opus-mt-en-de",
    ("es", "en"): "Helsinki-NLP/opus-mt-es-en",
    ("de", "en"): "Helsinki-NLP/opus-mt-de-en",
    ("es", "de"): "Helsinki-NLP/opus-mt-es-de",
    ("de", "es"): "Helsinki-NLP/opus-mt-de-es",
}

# Load translation model
def load_translation_model(src_lang, tgt_lang):
    model_name = TRANSLATION_MODELS.get((src_lang, tgt_lang))
    if model_name:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        return tokenizer, model
    return None, None

# Function to translate text
def translate_text(text, src_lang, tgt_lang):
    if src_lang == tgt_lang:
        return text  # No translation needed

    tokenizer, model = load_translation_model(src_lang, tgt_lang)
    if not model:
        return text  # Model not available for the selected pair

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Function to update UI text based on selected language
def update_language(*args):
    global current_language
    new_language = language_var.get()

    if new_language == current_language:
        return  # Avoid redundant updates

    src_lang = LANGUAGE_CODES[current_language]
    tgt_lang = LANGUAGE_CODES[new_language]

    # List of UI text elements
    texts_to_translate = [
        "Heart Disease Risk Application",
        "Enter your age:",
        "Enter your cholesterol reading:",
        "Enter your resting heart rate (in beats per minute):",
        "What is your biological gender?",
        "Have you ever smoked?",
        "Have you been diagnosed with exercise-induced angina?",
        "Submit"
    ]

    # Translate all UI elements in a batch
    translated_texts = [translate_text(text, src_lang, tgt_lang) for text in texts_to_translate]

    # Update UI with translated text
    label1.config(text=translated_texts[0])
    label2.config(text=translated_texts[1])
    label3.config(text=translated_texts[2])
    label4.config(text=translated_texts[3])
    label5.config(text=translated_texts[4])
    label6.config(text=translated_texts[5])
    label7.config(text=translated_texts[6])
    submit_button.config(text=translated_texts[7])

    # Set new language
    current_language = new_language

# Function to process responses and give an answer in the selected language
def process_response():
    try:
        age = int(age_entry.get())
        if age < 18 or age > 70:
            messagebox.showerror("Invalid input", "This assessment is based on data from 18 to 70 year olds.")
            return
        
        cholesterol = int(cholesterol_entry.get())
        if cholesterol > 240:
            messagebox.showerror("Danger for Heart Disease", "Danger of Heart Disease if cholesterol reading is over 240 mg/dL.")
            return
        
        resting_hr = int(resting_hr_entry.get())
        if resting_hr > 180: 
            messagebox.showerror("Danger for Heart Disease", "Danger of Heart Disease if resting heart rate reading is over 180 beats per minute.")
            return
        
        def risk_assessment(person_age, person_cholesterol, person_resting_hr, person_gender, person_smoking_status, person_exang_status):
            user_input = pd.DataFrame({'age': [person_age], 'cholesterol': [person_cholesterol], 'resting_hr': [person_resting_hr], 
                                       'gender': [person_gender], 'smoking_status': [person_smoking_status], 'exang_status' : [person_exang_status] })
            prediction = np.random.rand()
            #prediction = nn_model.predict(user_input)
            if prediction > 0.5:
                return 'Heart Disease Attack or Risk'
            else:
                return 'No Heart Disease Attack or Risk'

        result = risk_assessment(age, cholesterol, resting_hr)

        # Translate the result message into the selected language
        src_lang = "en"  # Default language of messages is English
        tgt_lang = LANGUAGE_CODES[current_language]
        translated_result = translate_text(result, src_lang, tgt_lang)

        messagebox.showinfo("Result", translated_result)

    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a valid numeric input.")

# Create the main application window
app = tk.Tk()
app.title("Heart Disease Risk Application")

# Default language
current_language = "English"

# Language Selection Dropdown
tk.Label(app, text="Select Language:").pack()
language_var = tk.StringVar(value="English")
language_dropdown = ttk.Combobox(app, textvariable=language_var, values=list(LANGUAGE_CODES.keys()))
language_dropdown.pack()
language_var.trace_add("write", update_language)

# Header
label1 = tk.Label(app, text="Heart Disease Risk Application")
label1.pack()

# Age Input
label2 = tk.Label(app, text="Enter your age:")
label2.pack()
age_entry = tk.Entry(app)
age_entry.pack()

# Cholesterol input
label3 = tk.Label(app, text="Enter your cholesterol reading:")
label3.pack()
cholesterol_entry = tk.Entry(app)
cholesterol_entry.pack()

# Resting Heart Rate input
label4 = tk.Label(app, text="Enter your resting heart rate (in beats per minute):")
label4.pack()
resting_hr_entry = tk.Entry(app)
resting_hr_entry.pack()

# Question 1: Gender request
label5 = tk.Label(app, text="What is your biological gender?")
label5.pack()
gender_entry = tk.StringVar()
tk.Radiobutton(app, text="M", variable=gender_entry, value="male").pack()
tk.Radiobutton(app, text="F", variable=gender_entry, value="female").pack()

# Question 2: Smoking status
label6 = tk.Label(app, text="Have you ever smoked?")
label6.pack()
smoking_status_entry = tk.StringVar()
tk.Radiobutton(app, text="Yes", variable=smoking_status_entry, value='yes').pack()
tk.Radiobutton(app, text="No", variable=smoking_status_entry, value='no').pack()

# Question 3: Exercise-induced angina status
label7 = tk.Label(app, text="Have you been diagnosed with exercise-induced angina?")
label7.pack()
exang_status_entry = tk.StringVar()
tk.Radiobutton(app, text="Yes", variable=exang_status_entry, value='yes').pack()
tk.Radiobutton(app, text="No", variable=exang_status_entry, value='no').pack()

# Submit Button
submit_button = tk.Button(app, text="Submit", command=process_response)
submit_button.pack()

# Run the application
app.mainloop()
